Importing netCDF4 library

In [1]:
import netCDF4 as nc

Importing Pandas for data manipulations

In [2]:
import pandas as pd

Importing Numpy for Speed Calculations

In [3]:
import numpy as np

Load your .netCDF4 file to a dataset variable

⚠️ **Alert: Dont forget to change the name of the saved file** 

In [4]:
dataset= nc.Dataset(r'C:\Users\user\Desktop\ERA5\Wind2022.nc')

Print information about the dataset

In [5]:
print(dataset)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    GRIB_centre: ecmf
    GRIB_centreDescription: European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre: 0
    Conventions: CF-1.7
    institution: European Centre for Medium-Range Weather Forecasts
    history: 2024-10-03T06:51 GRIB to CDM+CF via cfgrib-0.9.14.1/ecCodes-2.36.0 with {"source": "data.grib", "filter_by_keys": {"stream": ["oper"]}, "encode_cf": ["parameter", "time", "geography", "vertical"]}
    dimensions(sizes): valid_time(8760), latitude(4), longitude(6)
    variables(dimensions): int64 number(), int64 valid_time(valid_time), float64 latitude(latitude), float64 longitude(longitude), <class 'str'> expver(valid_time), float32 u10(valid_time, latitude, longitude), float32 v10(valid_time, latitude, longitude)
    groups: 


Check the variable list 

In [6]:
print(dataset.variables.keys())

dict_keys(['number', 'valid_time', 'latitude', 'longitude', 'expver', 'u10', 'v10'])


Retrieving these variables into separate lists

In [7]:
# Assuming the variable of interest is the wind component
# You might need to modify the variable name based on your file
u_wind = dataset.variables['u10'][:]
v_wind = dataset.variables['v10'][:]
time_var = dataset.variables['valid_time'][:]
lat = dataset.variables['latitude'][:]
lon = dataset.variables['longitude'][:]

Convert Time and Date to Humane Format

In [8]:
# Convert time to a human-readable format
time_units = dataset.variables['valid_time'].units
time_calendar = dataset.variables['valid_time'].calendar if hasattr(dataset.variables['valid_time'], 'calendar') else 'standard'
times = nc.num2date(time_var, units=time_units, calendar=time_calendar)

Choose the location 

In [9]:
# Select a specific location (e.g., at lat=40 and lon=70)
target_lat = 38.41
target_lon = 66.56

Coordinates to Index Coordinates

In [10]:
# Find the index of the closest latitude and longitude
lat_idx = (abs(lat - target_lat)).argmin()
lon_idx = (abs(lon - target_lon)).argmin()

In [11]:
print(f"Closest latitude: {lat[lat_idx]}")
print(f"Closest longitude: {lon[lon_idx]}")

Closest latitude: 38.33
Closest longitude: 66.56


Get the values of U and V components for this specific location and store in the list

In [12]:
# Extract data for the specific location
u_wind_data = u_wind[:, lat_idx, lon_idx]
v_wind_data = v_wind[:, lat_idx, lon_idx]

Calculate the WindSpeed according to the Formula 
$$
\text{Wind Speed} = \sqrt{u^2 + v^2}
$$

In [13]:
# Calculate wind speed using the formula: sqrt(u^2 + v^2)
wind_speed = np.sqrt(u_wind_data**2 + v_wind_data**2)

Preparing data for the CSV file creation

In [14]:
# Prepare the data for the CSV
data = {
    'year': [t.year for t in times],
    'month': [t.month for t in times],
    'day': [t.day for t in times],
    'hour': [t.hour for t in times],
    'u_wind': u_wind_data,
    'v_wind': v_wind_data,
    'wind_speed': wind_speed
}

Save the file

In [15]:
# Create a DataFrame
df = pd.DataFrame(data)


⚠️ **Alert: Dont forget to change the name of the saved file** 

In [16]:
# Save to CSV
df.to_csv('wind_data_2022.csv', index=False)